In [1]:
import pandas as pd
import numpy as np
import requests
import re

In [14]:
links = []

for i in range(12):
    if i == 0:
        response = requests.get('https://novosibirsk.cian.ru/cat.php?deal_type=sale&engine_version=2&location%5B0%5D=201245&offer_type=flat&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1')
    else:
        link = f'https://novosibirsk.cian.ru/cat.php?deal_type=sale&engine_version=2&location%5B0%5D=201245&offer_type=flat&p={i}&room1=1&room2=1&room3=1&room4=1&room5=1&room6=1&room7=1&room9=1'
        response = requests.get(link)
    html_main = response.text 
    links += re.findall(r'https://novosibirsk.cian.ru/sale/flat/[0-9]{9}', html_main)

In [15]:
links = set(links)